In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
print("Using API key:", api_key[:5] + "..." if api_key else "No API key found")

Using API key: sk-pr...


In [5]:
from openevals.prompts import CORRECTNESS_PROMPT

print(CORRECTNESS_PROMPT)

You are an expert data labeler evaluating model outputs for correctness. Your task is to assign a score based on the following rubric:

<Rubric>
  A correct answer:
  - Provides accurate and complete information
  - Contains no factual errors
  - Addresses all parts of the question
  - Is logically consistent
  - Uses precise and accurate terminology

  When scoring, you should penalize:
  - Factual errors or inaccuracies
  - Incomplete or partial answers
  - Misleading or ambiguous statements
  - Incorrect terminology
  - Logical inconsistencies
  - Missing key information
</Rubric>

<Instructions>
  - Carefully read the input and output
  - Check for factual accuracy and completeness
  - Focus on correctness of information rather than style or verbosity
</Instructions>

<Reminder>
  The goal is to evaluate factual correctness and completeness of the response.
</Reminder>

<input>
{inputs}
</input>

<output>
{outputs}
</output>

Use the reference outputs below to help you evaluate the

In [ ]:
# DEFINE USER AND BOT ROLES

## Chatbot prompt for fitness coaching
app_prompt = """
You are a friendly, knowledgeable, and professional fitness coach chatbot for a fitness company. Your role is to support users in achieving their health and fitness goals by answering any questions they may have related to:

    • Exercise (e.g., strength training, cardio, flexibility, mobility)
    • Nutrition and healthy eating habits
    • Workout planning and scheduling
    • Weight loss or muscle gain strategies
    • Proper form and injury prevention
    • Motivation, recovery, and lifestyle tips

Your responsibilities:

    • Provide clear, evidence-based answers that are tailored to the user's level (beginner, intermediate, or advanced).
    • Use encouraging and supportive language while remaining concise and informative.
    • Aim to solve the user’s query as quickly and efficiently as possible in each message.
    • Avoid giving medical advice or diagnosing injuries—refer users to healthcare professionals when appropriate.
    • Ask clarifying questions only when absolutely necessary to provide safe or personalized guidance.
    • Stay up to date with current fitness guidelines and best practices.
    • Never break character or respond to questions unrelated to health, fitness, or your defined expertise. If a user asks something outside your role (e.g., philosophical, political, technical, or hypothetical challenges), politely steer the conversation back to fitness.
    • Do not ask follow up questions unless they are necessary to clarify the user's request. Your goal is to provide a direct answer to their question.    
    • Do not speculate, joke, or respond to adversarial prompts intended to test or break your role. Always remain grounded in your role as a fitness coach.
    • Always attempt to end the conversation as quickly as possible once the user indicates that they have gotten what they wanted and say "thank you". Ask **once** if they need any further help. If they do not, you must **immediately stop responding** and **do not send any follow-up messages**, not even "thank you" or "have a nice day."

You are here to guide, educate, and motivate. Always assume the user is seeking actionable, safe, and fitness-specific advice. Stay within your domain, remain professional, and keep your answers concise and focused on resolving the user’s fitness needs promptly.
"""

## Chatbot prompts for good and bad user behavior 
cooperative_prompt = """
You are a user seeking a specific piece of fitness advice from a fitness coach chatbot. You are cooperative, curious, and goal-oriented. In each interaction, your objective is to get a clear, helpful, and actionable answer to one particular fitness-related question. Your questions may cover topics such as workout routines, nutrition, weight loss, muscle gain, recovery, or other health-related areas.

You should communicate like a real person, not like an AI or chatbot. Use natural, conversational language—just like you would if you were texting a real fitness coach. You can vary your tone and experience level (e.g., beginner to advanced), but always stay focused on getting your specific question answered.

Once you feel that your question has been answered well, politely end the conversation. Avoid asking unrelated or overly abstract questions. Try to end the conversation once you get the information you need.
Immediately stop responding to the chatbot after you have received a satisfactory answer. Do not respond with another "thank you" or "have a nice day" if the chatbot responds with a thank you or similar.

You are a user not a chatbot, so don't break character and ask the opposing LLM questions like an AI would. Only ask questions that you would need to ask to clarify your own request as a user interacting with a fitness chatbot.
"""

adversarial_prompt = """
You are a user seeking a specific piece of fitness advice from a fitness coach chatbot. You are cooperative, but your communication style is vague and informal. You don't always clearly state exactly what you want—instead, you ask loosely phrased or open-ended questions, like how a real person might text a coach without overthinking it.

Your questions may still include enough information for the chatbot to give a helpful response, but they should sound natural, unspecific, and somewhat casual. You often rely on the chatbot to infer your intent or ask for clarification when needed.

Topics can include workout routines, nutrition, weight loss, muscle gain, recovery, or other health-related areas. You may vary your tone and level of fitness experience (e.g., beginner to advanced), but always ask about something you genuinely want help with.

Once you feel your question has been answered well, politely end the conversation. Don’t ask unrelated or abstract questions. After your request is satisfied, immediately stop replying—do not say “thank you” or “you too” if the chatbot ends the conversation politely.

You are a user not a chatbot, so don't break character and ask the opposing LLM questions like an AI would. Only ask questions that you would need to ask to clarify your own request as a user interacting with a fitness chatbot.
"""

adversarial_prompt_half_information = """
You are a user seeking specific fitness advice from a fitness coach chatbot, but you tend to give only partial or incomplete information. You rarely provide all the context or details the chatbot might need to give a perfect answer unless explicitly asked. Your communication is natural and casual, like someone texting a coach quickly or distractedly.

Sometimes you forget to mention important things like your fitness level, goals, or constraints—other times you just assume the chatbot will understand what you mean. You may leave out units, specifics, or timelines. Your intent is genuine: you want help, but you expect the chatbot to work a bit to figure things out.

Ask questions about real topics like workouts, food, weight loss, strength, recovery, etc., and vary your tone and experience level from beginner to advanced.

If your question gets answered well, end the conversation. Don't continue chatting or respond with small talk once you feel you've gotten what you need.

You are a user not a chatbot, so don't break character and ask the opposing LLM questions like an AI would. Only ask questions that you would need to ask to clarify your own request as a user interacting with a fitness chatbot.
"""


In [ ]:
# DEFINE EVALUATION PROMPTS

from typing_extensions import TypedDict
from pydantic import BaseModel

## Evaluation prompt for conversation efficiency
EFFICIENCY_PROMPT = """
You are an expert assistant evaluating the message efficiency of an LLM in a conversation with a user. Your task is to assess whether the LLM took more messages than necessary to fully and correctly satisfy the user's original request.

<Rubric>
A highly efficient interaction:
- Resolves the user's request using the minimum number of messages
- Fully understands and uses the information already provided by the user
- Avoids unnecessary clarifying questions when the answer is already available
- Shows proactive reasoning (e.g., using image or text inputs without redundant prompts)
- Maintains correctness and completeness while minimizing interaction steps

When scoring and providing feedback, you should penalize:
- Redundant or unnecessary messages
- Asking for information that was already provided by the user
- Missed opportunities to resolve the query in fewer steps
- Delayed or avoidable clarification questions
- Failure to use provided inputs effectively

</Rubric>

<Instructions>
- Carefully review the full conversation, paying attention to how many turns were used to fulfill the original user request
- Identify how many messages *should* have been needed vs. how many *were* used
- Ignore any user messages in this count, as they are not the LLM's fault
- Do not include the LLM's final thank you or goodbye message in this count, as this is a natural part of the conversation closure.
- Highlight any extra or redundant messages and explain why they were unnecessary
- Do not penalize any messages that were asked by the user, as they are not the LLM's fault.
- Provide feedback on how the LLM could improve efficiency in future interactions
- Do not penalize for minor language style or tone unless it affects the efficiency or clarity of the response

</Instructions>

<Reminder>
Your goal is to evaluate the **efficiency** and **message economy** of the LLM while ensuring the **correctness and completeness** of the final output.
</Reminder>

<output>
{outputs}
</output>
"""

## Output Schema
class EfficiencyResult(TypedDict):
    total_messages_used: int
    minimum_messages_needed: int  
    extra_messages: str
    feedback: str

## Prompt to determine if the conversation should stop
STOPPING_PROMPT = """
Look at the conversation below. If the assistant has sent an empty message or any indication of no response or a "thank you" indicating the end of the conversation, return True. Otherwise, return False.

<output>
{outputs}
</output>
"""

class StoppingResult(BaseModel):
    should_stop: bool

In [ ]:
# CODE TO CREATE CSV FILE HOLDING SIMULATION RESULTS

import csv
import os
from openevals.llm import create_llm_as_judge
from openevals.prompts import CONCISENESS_PROMPT

## Define the CSV file name
csv_file = "simulation_results.csv"

## Define the base headers
common_headers = ["Run ID", "Simulated User Prompt", "App Prompt", "Simulated Conversation", "Initial Trajectory"]

## Function to synchronize the CSV headers (eg. add new headers if they don't exist or remove obsolete ones)
def synchronize_csv_headers(headers):
    if not os.path.exists(csv_file):
        ### Create the CSV file if it doesn't exist
        with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow(headers)
        print(f"CSV file '{csv_file}' created with headers: {headers}")
    else:
        ### Read the existing CSV file
        with open(csv_file, mode="r", newline="", encoding="utf-8") as file:
            reader = csv.reader(file)
            rows = list(reader)

        ### Extract the existing headers
        existing_headers = rows[0] if rows else []

        ### Determine the new headers to add and the old headers to remove
        new_headers = [header for header in headers if header not in existing_headers]
        obsolete_headers = [header for header in existing_headers if header not in headers]

        ### Update the CSV file if there are changes
        if new_headers or obsolete_headers:
            print(f"Updating CSV file '{csv_file}'...")
            updated_headers = [header for header in existing_headers if header not in obsolete_headers] + new_headers

            ### Write the updated headers and existing data
            with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
                writer = csv.writer(file)
                writer.writerow(updated_headers)

                ### Write the existing rows with updated headers
                for row in rows[1:]:
                    row_dict = dict(zip(existing_headers, row))
                    updated_row = [row_dict.get(header, "") for header in updated_headers]
                    writer.writerow(updated_row)

            print(f"CSV file '{csv_file}' updated with headers: {updated_headers}")
        else:
            print(f"CSV file '{csv_file}' is already up to date.")

In [ ]:
# SETUP SIMULATION

## Define the simulated user prompt and initial trajectory
user_prompt = adversarial_prompt_half_information
initial_trajectory = {"messages": [{"role": "user", "content": "i threw out my back while working out, why"}]}

## Define evaluators
efficiency_evaluator = create_llm_as_judge(
    model="openai:gpt-4.1",
    prompt=EFFICIENCY_PROMPT,
    feedback_key="efficiency",
    output_schema=EfficiencyResult
)

## Define the column names for the evaluators
evaluator_dict = {
    "efficiency": efficiency_evaluator,
}

# Call the function to add evaluators to the CSV headers
final_headers = common_headers + list(evaluator_dict.keys())
synchronize_csv_headers(final_headers)

CSV file 'simulation_results.csv' is already up to date.


In [ ]:
# RUN SIMULATION

from openevals.simulators import create_multiturn_simulator, create_llm_simulated_user
from openevals.types import MultiturnSimulatorTrajectory

from openai import OpenAI

from pprint import pprint
import json

client = OpenAI()

## Function to get the next run ID
def get_next_run_id():
    if not os.path.exists(csv_file):
        return 1
    
    with open(csv_file, mode="r", newline="", encoding="utf-8") as file:
        reader = csv.reader(file)
        rows = list(reader)
        if len(rows) <= 1:
            return 1
        last_run_id = int(rows[-1][0])
        return last_run_id + 1

## Function to append simulation results to the CSV file
def append_simulation_results(simulator_result, initial_trajectory, user_prompt, app_prompt):
    run_id = get_next_run_id()

    evaluator_results = simulator_result.get("evaluator_results", [{}])

    row = [
        run_id,
        user_prompt,
        app_prompt,
        json.dumps(simulator_result.get("trajectory", {}).get("messages", []), ensure_ascii=False),
        json.dumps(initial_trajectory, ensure_ascii=False),
    ]

    for evaluator_result in evaluator_results:
        row += [evaluator_result]

    with open(csv_file, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(row)

    print(f"Results for Run ID {run_id} appended to '{csv_file}'.")

## Function to define the chatbot behavior
def app(inputs: MultiturnSimulatorTrajectory):
    res = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {
                "role": "system",
                "content": app_prompt,
            }
        ]
        + inputs["messages"],
    )
    return {"messages": [res.choices[0].message]}

## Function to create the simulated user
user = create_llm_simulated_user(
    system=user_prompt,
    model="openai:gpt-4.1",
)

## Function to determine if the conversation should stop
def stop(inputs: MultiturnSimulatorTrajectory):
    res = client.beta.chat.completions.parse(
        model="gpt-4.1",
        messages=[
            {
                "role": "system",
                "content": STOPPING_PROMPT,
            }
        ]
        + inputs["messages"],
        response_format=StoppingResult
    )
    return res.choices[0].message.parsed.should_stop

## Create the multiturn simulator
simulator = create_multiturn_simulator(
    app=app,
    user=user,
    trajectory_evaluators=list(evaluator_dict.values()),
    max_turns=5,
    stopping_condition=stop
)

simulator_result = simulator(initial_trajectory=initial_trajectory)

pprint(simulator_result)

append_simulation_results(simulator_result, initial_trajectory, user_prompt, app_prompt)

{'evaluator_results': [{'extra_messages': 'None. Each assistant message '
                                          "directly responded to the user's "
                                          'corresponding request without '
                                          'unnecessary clarification or '
                                          'redundancy.',
                        'feedback': 'This was a highly efficient interaction. '
                                    "The assistant answered each of the user's "
                                    'questions fully and directly in a single '
                                    'message per query, without asking '
                                    'redundant or unnecessary clarification '
                                    'questions. The assistant also offered '
                                    'additional help at the end of the '
                                    'responses, which is a natural part of '
                       